In [4]:
import torch

# Load the model
checkpoint = torch.load('/home/cadt-02/Object-Detection/ros2_ws/model_- 31 january 2024 11_41.pt')
model = checkpoint['model']
model.eval()

ImportError: Matplotlib requires numpy>=1.20; you have 1.19.5

In [ ]:


try:
    # Convert the model's parameters to float data type
    model.float()
    
    input_tensor = torch.randn(1, 3, 640, 460).to(next(model.parameters()).device).type(model.parameters().__next__().dtype)
    torch.onnx.export(model, input_tensor, "model.onnx")
except RuntimeError as e:
    print(f"An error occurred: {str(e)}")


/home/cadt-02/.local/lib/python3.8/site-packages/ultralytics/nn/modules/head.py:53: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if self.dynamic or self.shape != shape:
/home/cadt-02/.local/lib/python3.8/site-packages/ultralytics/utils/tal.py:299: TracerWarning: Iterating over a tensor might cause the trace to be incorrect. Passing a tensor of different shape won't change the number of iterations executed (and might lead to errors or silently give incorrect results).
  for i, stride in enumerate(strides):


In [ ]:
import os
import tensorrt as trt
import pycuda.driver as cuda
import pycuda.autoinit

TRT_LOGGER = trt.Logger(trt.Logger.WARNING)

def get_engine(onnx_file_path, engine_file_path):
    # Check if the engine exists
    if os.path.exists(engine_file_path):
        # If it does, load the engine and return
        with open(engine_file_path, "rb") as f, trt.Runtime(TRT_LOGGER) as runtime:
            return runtime.deserialize_cuda_engine(f.read())
    else:
        # If it doesn't, we need to build the engine from the ONNX model.
        with trt.Builder(TRT_LOGGER) as builder, builder.create_network(1) as network, trt.OnnxParser(network, TRT_LOGGER) as parser:
            config = builder.create_builder_config()
            config.max_workspace_size = 1 << 28  # 256MiB
            builder.max_batch_size = 1
            # Parse the ONNX model
            with open(onnx_file_path, 'rb') as model:
                if not parser.parse(model.read()):
                    print('ERROR: Failed to parse the ONNX file.')
                    for error in range(parser.num_errors):
                        print(parser.get_error(error))
                    return None
            # Build and return an engine.
            engine = builder.build_engine(network, config)
            with open(engine_file_path, "wb") as f:
                f.write(engine.serialize())
            return engine

# Specify the paths to the ONNX model and the engine
onnx_file_path = 'model.onnx'
engine_file_path = 'model.trt'

# Get the engine
engine = get_engine(onnx_file_path, engine_file_path)

[03/06/2024-11:14:57] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.


In [ ]:
import os
import tensorrt as trt

TRT_MODEL_PATH = "/home/cadt-02/Object-Detection/model.trt"

# Check if the model file exists
if not os.path.exists(TRT_MODEL_PATH):
    raise FileNotFoundError(f"TensorRT model file not found: {TRT_MODEL_PATH}")

runtime = trt.Runtime(trt.Logger(trt.Logger.INFO))

with open(TRT_MODEL_PATH, "rb") as f:
    engine_data = f.read()

# Check if the engine data is not empty
if not engine_data:
    raise ValueError("The TensorRT model file is empty.")

engine = runtime.deserialize_cuda_engine(engine_data)

if engine is None:
    raise ValueError("Failed to deserialize the TensorRT engine.")

context = engine.create_execution_context()

# Set optimization profile dimensions
if engine.has_implicit_batch_dimension:
    raise ValueError("The TensorRT engine has an implicit batch dimension. This code is designed for explicit batch dimensions.")

if engine.num_optimization_profiles == 0:
    raise ValueError("The TensorRT engine does not have any optimization profiles defined.")

context.active_optimization_profile = 0

for binding in range(engine.num_bindings):
    if engine.binding_is_input(binding):
        input_name = engine.get_binding_name(binding)
        min_shape, opt_shape, max_shape = engine.get_profile_shape(0, input_name)
        print(f"Binding: {binding}, Optimal Shape: {opt_shape}")
        if not context.set_binding_shape(binding, opt_shape):
            raise ValueError(f"Failed to set the binding shape for input {input_name}.")

[03/14/2024-10:09:59] [TRT] [I] Loaded engine size: 43 MiB
[03/14/2024-10:09:59] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.
[03/14/2024-10:10:01] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +617, GPU +767, now: CPU 952, GPU 5348 (MiB)
[03/14/2024-10:10:01] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in engine deserialization: CPU +0, GPU +42, now: CPU 0, GPU 42 (MiB)
[03/14/2024-10:10:01] [TRT] [I] [MemUsageChange] Init cuDNN: CPU +0, GPU +20, now: CPU 952, GPU 5369 (MiB)
[03/14/2024-10:10:01] [TRT] [I] [MemUsageChange] TensorRT-managed allocation in IExecutionContext creation: CPU +0, GPU +34, now: CPU 0, GPU 76 (MiB)
Binding: 0, Optimal Shape: (1, 3, 640, 640)


/tmp/ipykernel_8277/670972004.py:33: DeprecationWarning: Use set_optimization_profile_async instead.
  context.active_optimization_profile = 0
/tmp/ipykernel_8277/670972004.py:36: DeprecationWarning: Use get_tensor_mode instead.
  if engine.binding_is_input(binding):
/tmp/ipykernel_8277/670972004.py:37: DeprecationWarning: Use get_tensor_name instead.
  input_name = engine.get_binding_name(binding)
/tmp/ipykernel_8277/670972004.py:38: DeprecationWarning: Use get_tensor_profile_shape instead.
  min_shape, opt_shape, max_shape = engine.get_profile_shape(0, input_name)
/tmp/ipykernel_8277/670972004.py:40: DeprecationWarning: Use set_input_shape instead.
  if not context.set_binding_shape(binding, opt_shape):


In [2]:
import tensorrt as trt

print("TensorRT version: ")
print(trt.__version__)
import torch
import onnx

print("PyTorch version: ")
print(torch.__version__)

print("ONNX version: ")
print(onnx.__version__)


TensorRT version: 
8.5.2.2


/home/cadt-02/.local/lib/python3.8/site-packages/torch/package/_mock_zipreader.py:17: UserWarning: Failed to initialize NumPy: module compiled against API version 0xe but this version of numpy is 0xd (Triggered internally at  ../torch/csrc/utils/tensor_numpy.cpp:67.)
  _dtype_to_storage = {data_type(0).dtype: data_type for data_type in _storages}
/home/cadt-02/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PyTorch version: 
1.9.0
ONNX version: 
1.15.0


In [3]:
import tensorrt as trt
import os

TRT_MODEL_PATH = "/home/cadt-02/Object-Detection/model.trt"

# Check if the model file exists
if not os.path.exists(TRT_MODEL_PATH):
    raise ValueError(f"Model file {TRT_MODEL_PATH} does not exist")

# Load the TensorRT runtime and create an engine and context
runtime = trt.Runtime(trt.Logger(trt.Logger.WARNING))
with open(TRT_MODEL_PATH, 'rb') as f:
    engine_data = f.read()

if engine_data is None:
    raise ValueError("Failed to read TensorRT engine data")

engine = runtime.deserialize_cuda_engine(engine_data)

if engine is None:
    raise ValueError("Failed to deserialize TensorRT engine")

context = engine.create_execution_context()

if context is None:
    raise ValueError("Failed to create TensorRT execution context")device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print("Successfully initialized TensorRT model")

[03/18/2024-10:44:55] [TRT] [W] Using an engine plan file across different models of devices is not recommended and is likely to affect performance or even cause errors.
Successfully initialized TensorRT model


In [5]:
import torch

print("CUDA available:", torch.cuda.is_available())
print("CUDA device count:", torch.cuda.device_count())

CUDA available: True
CUDA device count: 1


In [1]:
import torch
import torchvision

print("PyTorch version:", torch.__version__)
print("Torchvision version:", torchvision.__version__)

PyTorch version: 2.1.0a0+41361538.nv23.06
Torchvision version: 0.16.1+fdea156


In [1]:
import torch


print("PyTorch version:", torch.__version__)

print("CUDA version:", torch.version.cuda)

PyTorch version: 2.1.0a0+41361538.nv23.06
CUDA version: 11.4


In [2]:
import torch
import torchvision
import torchvision.transforms as transforms

# 1. Check torchvision version
print(torchvision.__version__)

# 2. Prepare your dataset
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Add this line
    transforms.ToTensor()
])
trainset = torchvision.datasets.CIFAR10(root='./model_- 18 march 2024 15_19.pt', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4, shuffle=True)

# 3. Define your model architecture
model = torchvision.models.resnet18(pretrained=False)

# 4. Define your loss function and optimizer
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# 5. Train your model
for epoch in range(2):  # loop over the dataset multiple times
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

# 6. Save your model
MODEL_PATH = './model.pt'
torch.save(model, MODEL_PATH)

ModuleNotFoundError: No module named 'torch._custom_ops'

In [6]:
#Importing Libraries
import cv2
import supervision as sv
from ultralytics import YOLO

def live_object_detection():
    '''main Function'''
    # Initialize the camera
    cap = cv2.VideoCapture(0)
    model = YOLO("/home/cadt-02/Object-Detection/ros2_ws/model_- 18 march 2024 15_19.pt")

    box_annotator = sv.BoxAnnotator(
        thickness=2,
        text_thickness=2,
        text_scale=1
    )

    while True:
        ret, frame = cap.read()

        data = model(frame)
        detection = sv.Detections.from_ultralytics(data[0])

        label = [f"{model.model.names[ci]} {con:0.2f}"
                 for _, _, con, ci, _ in detection]

        if not ret:
            print("Error: Failed to grab frame.")
            break

        frame = box_annotator.annotate(scene=frame, detections=detection, labels=label)

        # Display the live camera feed with object detection annotations
        cv2.imshow("Live Object Detection", frame)

        # Exit the loop if the 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release the camera and close the OpenCV window
    cap.release()
    cv2.destroyAllWindows()

# Run the live object detection function
live_object_detection()

WARNING ⚠️ 'source' is missing. Using 'source=/home/cadt-02/.local/lib/python3.8/site-packages/ultralytics/assets'.


[ WARN:0@1008.397] global cap_v4l.cpp:997 open VIDEOIO(V4L2:/dev/video0): can't open camera by index
[ERROR:0@1008.400] global obsensor_uvc_stream_channel.cpp:159 getStreamChannelGroup Camera index out of range


RuntimeError: Couldn't load custom C++ ops. This can happen if your PyTorch and torchvision versions are incompatible, or if you had errors while compiling torchvision from source. For further information on the compatible versions, check https://github.com/pytorch/vision#installation for the compatibility matrix. Please check your PyTorch version with torch.__version__ and your torchvision version with torchvision.__version__ and verify if they are compatible, and if not please reinstall torchvision so that it matches your PyTorch install.